In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotions/text.csv


In [2]:
isKaggle = os.getenv('KAGGLE_KERNEL_RUN_TYPE')

In [3]:
from pathlib import Path

if isKaggle:
    path = Path('../input/emotions/text.csv')

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv(path)
df

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...,...
416804,416804,i feel like telling these horny devils to find...,2
416805,416805,i began to realize that when i was feeling agi...,3
416806,416806,i feel very curious be why previous early dawn...,5
416807,416807,i feel that becuase of the tyranical nature of...,3


In [6]:
vocab = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

# DataIngestion and Sampling


In [7]:
df.describe(include='object')

,text
count,416809
unique,393822
top,i feel more adventurous willing to take risks
freq,16


In [8]:
df = df.drop(columns = ['Unnamed: 0'])

In [9]:
df

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...
416804,i feel like telling these horny devils to find...,2
416805,i began to realize that when i was feeling agi...,3
416806,i feel very curious be why previous early dawn...,5
416807,i feel that becuase of the tyranical nature of...,3


In [10]:
sample = df.sample(n=1000)
sample

,text,label
113863,i do these things because i care about him and...,1
38132,i feel so alone at times when i think of this,0
344649,i international construction material amp indo...,1
33597,i feel strong in body but especially strong in...,1
244376,i understand why many people watch them the tr...,3
...,...,...
51573,i like that im keeping busy because it makes m...,1
136602,i want to meet want to talk but i feel a bit a...,4
89278,i would feel totally damaged,0
396586,i arrived at cp feeling a bit annoyed with the...,3


In [11]:
sample.describe(include ='all')

,text,label
count,1000,1000.000000
unique,1000,NaN
top,i feel contented sitting in the sun on my balc...,NaN
freq,1,NaN
mean,NaN,1.569000
std,NaN,1.498494
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,1.000000
75%,NaN,3.000000


In [12]:
sample.columns

Index(['text', 'label'], dtype='object')

In [13]:
sample = sample.rename(columns={'text': 'input'})

In [14]:
sample.iloc[1]

input    i feel so alone at times when i think of this
label                                                0
Name: 38132, dtype: object

In [15]:
sample.describe(include='all')

,input,label
count,1000,1000.000000
unique,1000,NaN
top,i feel contented sitting in the sun on my balc...,NaN
freq,1,NaN
mean,NaN,1.569000
std,NaN,1.498494
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,1.000000
75%,NaN,3.000000


In [16]:
sample['label'] = sample['label'].astype(float)

In [17]:
sample

,input,label
113863,i do these things because i care about him and...,1.0
38132,i feel so alone at times when i think of this,0.0
344649,i international construction material amp indo...,1.0
33597,i feel strong in body but especially strong in...,1.0
244376,i understand why many people watch them the tr...,3.0
...,...,...
51573,i like that im keeping busy because it makes m...,1.0
136602,i want to meet want to talk but i feel a bit a...,4.0
89278,i would feel totally damaged,0.0
396586,i arrived at cp feeling a bit annoyed with the...,3.0


# Tokenization
Transformers uses `Dataset` object for storing dataset 

In [18]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(sample)

In [19]:
ds

Dataset({
    features: ['input', 'label', '__index_level_0__'],
    num_rows: 1000
})

In [20]:
model = 'google-bert/bert-base-uncased'

AutoTokenizer will create a tokenizer appropriate for the model

In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertForSequenceClassification
tokz = AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [22]:
tokz

BertTokenizerFast(name_or_path='google-bert/bert-base-uncased', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [23]:
tokz.tokenize('i sometimes feel as though i make up reasons to be mad just because i still have that tasteless feeling growing inside of me')

['i',
 'sometimes',
 'feel',
 'as',
 'though',
 'i',
 'make',
 'up',
 'reasons',
 'to',
 'be',
 'mad',
 'just',
 'because',
 'i',
 'still',
 'have',
 'that',
 'taste',
 '##less',
 'feeling',
 'growing',
 'inside',
 'of',
 'me']

In [24]:
def tok_function(x): return tokz(x['input'])

In [25]:
tok_ds = ds.map(tok_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [26]:
tok_ds

Dataset({
    features: ['input', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [27]:
tok_ds[0]['input'], tok_ds[0]['input_ids']

('i do these things because i care about him and i want him to feel comfortable and hear about his trip',
 [101,
  1045,
  2079,
  2122,
  2477,
  2138,
  1045,
  2729,
  2055,
  2032,
  1998,
  1045,
  2215,
  2032,
  2000,
  2514,
  6625,
  1998,
  2963,
  2055,
  2010,
  4440,
  102])

In [28]:
# Getting token of `1` 
tokz.vocab['1']

1015

In [29]:
tok_ds.rename_columns({'label': 'labels'})

Dataset({
    features: ['input', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

## Splitting Data

In [30]:
# Transformers uses DatasetDict to store validation and training sets
dds = tok_ds.train_test_split(0.1, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['input', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 900
    })
    test: Dataset({
        features: ['input', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100
    })
})

## Setting Metric

In [31]:
# Correlation Coefficient
def corr(x,y): return np.corrcoef(x,y)[0][1]

In [32]:
#np.corrcoef(sample['input'], sample['label'])

Transformers expects metric to be returned as dict. 

In [33]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

# Training

In [34]:
from transformers import TrainingArguments, Trainer

2024-02-18 20:16:57.845767: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-18 20:16:57.845884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-18 20:16:58.014439: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [35]:
bs = 128
epoch = 4
lr = 8e-5

In [36]:
# TrainingArgument class is used to set up the arguments
args = TrainingArguments('outputs', learning_rate=lr, 
                         warmup_ratio=0.1, 
                         lr_scheduler_type='cosine', 
                         fp16=True,
                         evaluation_strategy='epoch',
                         weight_decay=0.01, 
                         report_to='none')

In [37]:
model = AutoModelForSequenceClassification.from_pretrained(model, num_labels =1)
trainer = Trainer(model, 
                  args, 
                  train_dataset=dds['train'],
                  eval_dataset=dds['test'],
                  tokenizer=tokz, 
                  compute_metrics = corr_d)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
"""model = BertForSequenceClassification.from_pretrained(model, num_labels =1)
trainer = Trainer(model, 
                  args, 
                  train_dataset=dds['train'],
                  eval_dataset=dds['test'],
                  tokenizer=tokz,
                  compute_metrics=corr_d)"""

"model = BertForSequenceClassification.from_pretrained(model, num_labels =1)\ntrainer = Trainer(model, \n                  args, \n                  train_dataset=dds['train'],\n                  eval_dataset=dds['test'],\n                  tokenizer=tokz,\n                  compute_metrics=corr_d)"

In [39]:
trainer.train();

Epoch,Training Loss,Validation Loss


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 100

In [ ]:
print(dds['train'].features['label'].dtype)
print(dds['test'].features['label'].dtype)

# Prediction

In [ ]:
preds = trainer.predict('I\'m satisfied with my progress but I can work more faster')
preds